# Change Detection Experiment

In [2]:
import ee
import geemap
from geemap import *
import json
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import os

# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
#ee.Initialize()

# !pip install geemap


In [3]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

## Data Preparation

### Import and Visualise AOI GeoJSON

In [4]:
#clip image to only include aoi region
file_path = os.path.abspath('/Users/joycelynlongdon/Desktop/Cambridge/CambridgeCoding/MRES/Data/GeoJSONS/PIREDD_mai_ndombe.geojson')
with open(file_path) as f:
    studyRegion = json.load(f)

studyRegion = ee.FeatureCollection(studyRegion).first().geometry()
print(studyRegion)
Map.setCenter(18.4276, -2.6357,7)
Map.addLayer(studyRegion,{},"studyRegion")
Map

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Element.geometry",
    "arguments": {
      "feature": {
        "functionInvocationValue": {
          "functionName": "Collection.first",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection",
                "arguments": {
                  "features": {
                    "arrayValue": {
                      "values": [
                        {
                          "functionInvocationValue": {
                            "functionName": "Feature",
                            "arguments": {
                              "geometry": {
                                "functionInvocationValue": {
                                  "functionName": "GeometryConstructors.Polygon",
                                  "arguments": {
                                    "coordinates": {
                                      "constantValu

Map(bottom=1851.0, center=[-2.6357, 18.4276], controls=(WidgetControl(options=['position'], widget=HBox(childr…

### Create Image Collection and Cloud Mask



In [7]:
point = ee.Geometry.Point(18.4276, -2.6357)

#define cloud free composite




#define cloud mask
def cloudMask(image):
  #Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    #Get the pixel QA band.
    qa = image.select('pixel_qa')
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)
  
    
#grab image collection for the required date
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2013-01-01', '2013-12-31')
cloud_free_image = image.map(cloudMask)#apply cloud mask
median_image = cloud_free_image.median().select(['B1','B2','B3','B4','B5','B6','B7','B10','B11','pixel_qa'])#take median pixel values for all bands
monthly_image = median_image.clip(studyRegion)#clip to studyRegion

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'] #RGB Composite
}

Map.addLayer(monthly_image, vis_params, "Landsat-8_2016")
Map

Map(bottom=16924.0, center=[-2.6357, 18.4276], controls=(WidgetControl(options=['position'], widget=HBox(child…

In [14]:
print(image)

ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.map",
    "arguments": {
      "baseAlgorithm": {
        "functionDefinitionValue": {
          "argumentNames": [
            "_MAPPING_VAR_0_0"
          ],
          "body": {
            "functionInvocationValue": {
              "functionName": "Image.updateMask",
              "arguments": {
                "image": {
                  "argumentReference": "_MAPPING_VAR_0_0"
                },
                "mask": {
                  "functionInvocationValue": {
                    "functionName": "Image.and",
                    "arguments": {
                      "image1": {
                        "functionInvocationValue": {
                          "functionName": "Image.eq",
                          "arguments": {
                            "image1": {
                              "functionInvocationValue": {
                                "functionName": "Image.bitwiseAnd",
      

### Reduce the cloud free imageCollection into a multiband image and clip to studyRegion

In [8]:
#reduce image collection into one multiband image taking the median value of each pixel
image = image.median()
#clip image to required boundary for studyRegion
image = image.clip(studyRegion)

Map.setCenter(18.4276, -2.6357,7)

Map.addLayer(image, vis_params, "Landsat-8_2016")
Map

Map(bottom=16924.0, center=[-2.6357, 18.4276], controls=(WidgetControl(options=['position'], widget=HBox(child…

In [19]:
#clip image to only include aoi region
file_path_1 = os.path.abspath('/Users/joycelynlongdon/Desktop/Cambridge/CambridgeCoding/MRES/GEE_examples/Input_Data/mai_ndombe.json')
file_path_2 = os.path.abspath('/Users/joycelynlongdon/Desktop/Cambridge/CambridgeCoding/MRES/GEE_examples/Input_Data/mai_ndombe_copy.json')

file_path = (file_path_1, file_path_2)

def open_aoi_file(file_path):
    with open(file_path) as f:
        aoi = json.load(f)
        aoi = ee.FeatureCollection(studyRegion).first().geometry()
    return aoi

for i in file_path:
    open_aoi_file(file_path)
    if aoi = aoi_train
    elif file_path == file_path[1]
        open_aoi_file(file_path[0])
        aoi = aoi_test

"""
studyRegion = ee.FeatureCollection(studyRegion).first().geometry()
Map.setCenter(18.4276, -2.6357,7)
Map.addLayer(studyRegion,{},"studyRegion")
Map
"""

SyntaxError: invalid syntax (<ipython-input-19-32e81a65a48b>, line 14)

### Inspect Metadata

In [7]:
# Get information about the bands as a list.
band_names = image.bandNames()
print('Band names:', band_names.getInfo())  # ee.List of band names

Band names: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'sr_aerosol', 'pixel_qa', 'radsat_qa']


In [8]:
# Get a list of all metadata properties.
properties = image.propertyNames()
print('Metadata properties:',
      properties.getInfo())  # ee.List of metadata properties

Metadata properties: ['system:footprint', 'system:bands', 'system:band_names']


In [9]:
# Get a specific metadata property.
cloudiness = image.get('CLOUD_COVER')
print('CLOUD_COVER:', cloudiness.getInfo())  # ee.Number


CLOUD_COVER: None


## Change Detection

### Define Parameters

In [26]:
#change detection parameters
breakpointBands = ['GREEN','RED','NIR','SWIR1','SWIR2'] #bands to be used for change detection (if unspecififed, all bands are used)
tmaskBands = ['GREEN','SWIR2'] #bands to use for iterative TMask cloud detection which are typically green and swir2

#all other parameters are kept as default for now
#minObservations = 6 #the number of observations required to flag a change
#chiSquareProbability  = .99 #the chi-squared probability threshold for change detection in the range [0,10]
#dateFormat = 0 #julian days
#minNumOfYearsScaler = 1.0 #minimum number of years to appply new fitting

#lamda = 20/10000, #lambda for LASSO regression fitting
#maxIterations = 25000



### Implementing the CCDC algorithm 

The Output bands are:
- tStart: The start date of each model segment.
- tEnd: The end date of each model segment.
- tBreak: The model break date if a change is detected.
- numObs: The number of observations used in each model segment.
- changeProb: A numeric value representing the multi-band change probability.
- *_coefs: The regression coefficients for each of the bands in the image collection.
- *_rmse: The model root-mean-square error for each segment and input band.
- *_magnitude: For segments with changes detected, this represents the normalized residuals during the change period.



In [30]:
change_detection = ee.Algorithms.TemporalSegmentation.Ccdc(breakpointBands,tmaskBands)
#print(change_detection)

## Classification

### Formatting training data to be used in classification.

There are a few requirements for the training data for it to work with the rest of the process.
- The data must be an Earth Engine FeatureCollection of point geometries.
- An attribute in each point must contain a numeric value indicating the associated land cover.
- An attribute in each point must contain a year that corresponds to the land cover label.